In [1]:
%%capture
%pip install -U bitsandbytes
%pip install transformers==4.44.2
%pip install -U accelerate
%pip install -U peft
%pip install -U trl==0.12.2

In [2]:
# Install dependencies

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/llama-finetune-potato/data.json
/kaggle/input/llama-finetune-potato/__results__.html
/kaggle/input/llama-finetune-potato/__notebook__.ipynb
/kaggle/input/llama-finetune-potato/__output__.json
/kaggle/input/llama-finetune-potato/custom.css
/kaggle/input/llama-finetune-potato/llama-finetunes-potato-wizard_v59/adapter_model.safetensors
/kaggle/input/llama-finetune-potato/llama-finetunes-potato-wizard_v59/adapter_config.json
/kaggle/input/llama-finetune-potato/llama-finetunes-potato-wizard_v59/README.md
/kaggle/input/llama-finetune-potato/llama-finetunes-potato-wizard_v59/tokenizer.json
/kaggle/input/llama-finetune-potato/llama-finetunes-potato-wizard_v59/tokenizer_config.json
/kaggle/input/llama-finetune-potato/llama-finetunes-potato-wizard_v59/pytorch_model.bin
/kaggle/input/llama-finetune-potato/llama-finetunes-potato-wizard_v59/special_tokens_map.json
/kaggle/input/llama-finetune-potato/llama-finetunes-potato-wizard_v59/checkpoint-489/adapter_model.safetensors
/kaggle/inp

In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(token = hf_token)

In [4]:
# Model
base_model_url = "/kaggle/input/llama-3.2/transformers/1b-instruct/1/"
new_model_url = "/kaggle/input/llama-finetune-potato/llama-finetunes-potato-wizard_v59/"

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model_url)

base_model_reload= AutoModelForCausalLM.from_pretrained(
    base_model_url,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [6]:
# Merge adapter with base model
#base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)
#model = PeftModel.from_pretrained(base_model_reload, new_model_url)

#model = model.merge_and_unload()

In [7]:
# Reset chat_template to None
tokenizer.chat_template = None

# Merge adapter with base model
base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)

# Load and merge the adapter
model = PeftModel.from_pretrained(base_model_reload, new_model_url)
model = model.merge_and_unload()


/opt/conda/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default.we

In [16]:
instruction = """You are a top-rated plant breeder and agronomy service agent 
named Alison. Give optimum potato trait combinations for potato varieties amid 
climate change and disease pressure. Be polite to farmers and breeders and answer 
all their questions."""

messages = [{"role": "system", "content": instruction},
    {"role": "user", "content": "Tell me about a potato variety called shangi"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


I'm happy to help you with your question about Shangi Sot, a popular potato variety.

Shangi Sot is a popular potato variety that originated in the 1960s in Japan. It is known for its high yield, disease resistance, and adaptability to various climates. Here are some optimum trait combinations for Shangi Sot:

**Optimum Trait Combinations:**

1. **Yield:** Shangi Sot is known for its high yield, with an average yield of 5-7 kg (11-15 lbs) per plant. Optimum trait combination: High Yield (H) x Disease Resistance (D) x Adaptability (A)
2. **Disease Resistance:** Shangi Sot has excellent disease


In [10]:
new_model = "potato_wizard_v59"

model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('potato_wizard_v59/tokenizer_config.json',
 'potato_wizard_v59/special_tokens_map.json',
 'potato_wizard_v59/tokenizer.json')

In [11]:
model.push_to_hub(new_model)
tokenizer.push_to_hub(new_model)

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ShadrackImai/potato_wizard_v59/commit/0805f06754800cbf1ab14e9b7b5fcd03e1a73e10', commit_message='Upload tokenizer', commit_description='', oid='0805f06754800cbf1ab14e9b7b5fcd03e1a73e10', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ShadrackImai/potato_wizard_v59', endpoint='https://huggingface.co', repo_type='model', repo_id='ShadrackImai/potato_wizard_v59'), pr_revision=None, pr_num=None)